In [ ]:
import boto3
import random

s3client = boto3.client('s3')

In [ ]:
# Bucket names are globally unique. Here we'll add a random element to the name to increase the odds of
# avoiding a name collision
bucket_name = 'ds-e1-' + str(random.randint(1, 9999999))
print bucket_name

In [ ]:
response = s3client.create_bucket(
    Bucket=bucket_name
)

print response

In [ ]:
# Execute this cell to enable versioning. Once versioning is enable for a bucket it cannot be removed,
# although it can be disabled.
response = s3client.put_bucket_versioning(
    Bucket=bucket_name,
    VersioningConfiguration={
        'MFADelete': 'Disabled',
        'Status': 'Enabled'
    }
)

print response

In [ ]:
response = s3client.get_bucket_versioning(
    Bucket=bucket_name
)

print response

response = s3client.get_bucket_logging(
    Bucket=bucket_name
)

print response

response = s3client.get_bucket_acl(
    Bucket=bucket_name
)

print response

# returns an error if no bucket lifecycle configured
#response = s3client.get_bucket_lifecycle_configuration(
#    Bucket=bucket_name
#)

#print response

In [ ]:
f = open("./hello.txt","w")
f.write("hello, world")
f.close()

In [ ]:
# Let's write the file content to the bucket specifying the reduced redundancy class. Had 
# we omitted the StorageClass it would have defaulted to STANDARD. 

s3 = boto3.resource('s3')
with open('./hello.txt', 'r') as f:
    read_data = f.read()
    response = s3.Object(bucket_name, 'hello.txt').put(Body=read_data, StorageClass='REDUCED_REDUNDANCY')
    print response

# upload_file defaults the storage to standard
#s3 = boto3.resource('s3')
#response = s3.meta.client.upload_file('./hello.txt', bucket_name, 'hello.txt')
#print response

In [ ]:
s3 = boto3.resource('s3')
object_acl = s3.ObjectAcl(bucket_name,'hello.txt')
response = object_acl.put(
    ACL='public-read'
)

print response

In [ ]:
f = open("./hello.txt","w")
f.write("hello, world -- again")
f.close()

In [ ]:
# Note that a single S3 put is limited to a max size of 5 gigabytes, and 
# that AWS recommends using multipart uploads when object size gets around 100 MB.
# 

s3 = boto3.resource('s3')
response = s3.meta.client.upload_file('./hello.txt', bucket_name, 'hello.txt')
print response

# S3 has Read after Write consistency semantics after the PUT of a new object, and
# eventual consistency after PUTs that overwrite and DELETEs

In [ ]:
s3 = boto3.resource('s3')
object_acl = s3.ObjectAcl(bucket_name,'hello.txt')
response = object_acl.put(
    ACL='public-read'
)

print response

In [ ]:
response = s3client.list_object_versions(
    Bucket=bucket_name,
    Prefix='hello.txt',
    MaxKeys=100
)


versions = response['Versions']
for version in versions:
    print "key: '{}' version {}".format(version['Key'], version['VersionId'])


In [ ]:
# This one does a delete on the object. In the console you can see the deleted versions and marker
response = s3client.delete_object(
        Bucket=bucket_name,
        Key='hello.txt'
)

print response

In [ ]:
response = s3client.list_object_versions(
    Bucket=bucket_name,
    Prefix='hello.txt',
    MaxKeys=100
)


print response

if 'Versions' in response:
    versions = response['Versions']
    for v in versions:
        print v['Key']
else:
    print 'No versions'
    


In [ ]:
if 'DeleteMarkers' in response:
    deleteMarkers = response['DeleteMarkers']
    for dm in deleteMarkers:
        print dm
else:
    print 'No delete markers'

In [ ]:
# This cell deletes all the object versions in the bucket. There's no restore/delete marker after this.
for version in versions:
    response = s3client.delete_object(
        Bucket=bucket_name,
        Key=version['Key'],
        VersionId=version['VersionId']
    )

    print response

In [ ]:
# This cell deletes all the delete markers in the bucket. If you delete an object (just the object, not all the 
# object versions( then delete the deletemarker, this 'restores' the object.
response = s3client.list_object_versions(
    Bucket=bucket_name,
    Prefix='hello.txt',
    MaxKeys=100
)

if 'DeleteMarkers' in response:
    deleteMarkers = response['DeleteMarkers']
    for dm in deleteMarkers:
        response = s3client.delete_object(
            Bucket=bucket_name,
            Key=dm['Key'],
            VersionId=dm['VersionId']
        )
        
        print response
else:
    print 'No delete markers to delete'

In [ ]:
response = s3client.delete_bucket(
    Bucket=bucket_name
)


print response